# FAUST Annotation Embedding

In this notebook we are generating an annotation-driven embedding using FAUST annotations of the MMRF study.

In [1]:
import jscatter
import numpy as np
import pandas as pd
from cmaps import glasbey_dark

## Data Loading

In [9]:
data = pd.read_csv('data/MMRF_1267_output.csv')
data.head(5)

,umapX,umapY,sampleOfOrigin,faustLabels,CD3,HLADR,NKG2D,CD8,CD45,CD4,...,CD56_faust_annotation,KLRG1_faust_annotation,Ki67_faust_annotation,PDL1_faust_annotation,Tbet_faust_annotation,ICOS_faust_annotation,NKG2A_faust_annotation,CD138_faust_annotation,CD1c_faust_annotation,DNAM1_faust_annotation
0,1.538682,0.374683,MMRF_1267.fcs,CD3-HLADR+NKG2D-CD8-CD45+CD4-CD5-CD14-CD19+CD1...,1.075370,5.342214,0.000000,0.000268,4.862887,0.824602,...,-,-,-,-,-,-,-,-,+,-
1,-3.134375,11.461645,MMRF_1267.fcs,0_0_0_0_0,5.824924,0.915367,3.522006,4.770885,4.734933,0.000000,...,-,+,-,-,-,-,-,-,-,+
2,-0.935311,12.303033,MMRF_1267.fcs,0_0_0_0_0,5.358062,1.360134,1.814536,2.393424,4.235237,0.000000,...,-,-,+,-,+,+,-,-,+,+
3,2.128425,-5.859528,MMRF_1267.fcs,CD3-HLADR-NKG2D-CD8-CD45+CD4-CD5-CD14-CD19-CD1...,0.000000,2.271876,0.347916,0.000000,1.779969,0.547474,...,-,-,-,-,-,-,-,-,-,-
4,-2.191941,13.085436,MMRF_1267.fcs,0_0_0_0_0,5.449534,0.000000,2.883831,4.385998,4.845957,0.000000,...,-,+,-,-,-,-,-,+,-,-


## Data Preparation

We'll start by extracting the "raw" marker expression values

In [5]:
markers = ['CD3', 'HLADR', 'NKG2D', 'CD8', 'CD45', 'CD4', 'CD5', 'CD14', 'CD19', 'CD127', 'CD11b', 'CD25', 'CD45RO', 'CD33', 'CD38', 'CD27', 'CD11c', 'CD45RA', 'CD66b', 'CCR7', 'GranzymeB', 'CD39', 'CD28', 'PD1', 'CD57', 'CD123', 'CD16', 'TIGIT', 'TIM3__CD366_', 'CD56', 'KLRG1', 'Ki67', 'PDL1', 'Tbet', 'ICOS', 'NKG2A', 'CD138', 'CD1c', 'DNAM1']
raw_expression = mmrf_1267_embedding[markers].values

Next we create a new column for the _complete_ FAUST annotation labels. In comparison to `faustLabels`, the complete FAUST label does not depent on selected phenotypes and is just the concatenation of all marker labels.

Think of the _complete FAUST label_ as the cluster name that each data entry belongs to.

In [10]:
data['complete_faust_label'] = 'CD3' + data['CD3_faust_annotation'] + 'HLADR' + data['HLADR_faust_annotation'] + 'NKG2D' + data['NKG2D_faust_annotation'] + 'CD8' + data['CD8_faust_annotation'] + 'CD45' + data['CD45_faust_annotation'] + 'CD4' + data['CD4_faust_annotation'] + 'CD5' + data['CD5_faust_annotation'] + 'CD14' + data['CD14_faust_annotation'] + 'CD19' + data['CD19_faust_annotation'] + 'CD127' + data['CD127_faust_annotation'] + 'CD11b' + data['CD11b_faust_annotation'] + 'CD25' + data['CD25_faust_annotation'] + 'CD45RO' + data['CD45RO_faust_annotation'] + 'CD33' + data['CD33_faust_annotation'] + 'CD38' + data['CD38_faust_annotation'] + 'CD27' + data['CD27_faust_annotation'] + 'CD11c' + data['CD11c_faust_annotation'] + 'CD45RA' + data['CD45RA_faust_annotation'] + 'CD66b' + data['CD66b_faust_annotation'] + 'CCR7' + data['CCR7_faust_annotation'] + 'GranzymeB' + data['GranzymeB_faust_annotation'] + 'CD39' + data['CD39_faust_annotation'] + 'CD28' + data['CD28_faust_annotation'] + 'PD1' + data['PD1_faust_annotation'] + 'CD57' + data['CD57_faust_annotation'] + 'CD123' + data['CD123_faust_annotation'] + 'CD16' + data['CD16_faust_annotation'] + 'TIGIT' + data['TIGIT_faust_annotation'] + 'TIM3__CD366_' + data['TIM3__CD366__faust_annotation'] + 'CD56' + data['CD56_faust_annotation'] + 'KLRG1' + data['KLRG1_faust_annotation'] + 'Ki67' + data['Ki67_faust_annotation'] + 'PDL1' + data['PDL1_faust_annotation'] + 'Tbet' + data['Tbet_faust_annotation'] + 'ICOS' + data['ICOS_faust_annotation'] + 'NKG2A' + data['NKG2A_faust_annotation'] + 'CD138' + data['CD138_faust_annotation'] + 'CD1c' + data['CD1c_faust_annotation'] + 'DNAM1' + data['DNAM1_faust_annotation']
data[['faustLabels', 'complete_faust_label']].head(5)

,faustLabels,complete_faust_label
0,CD3-HLADR+NKG2D-CD8-CD45+CD4-CD5-CD14-CD19+CD1...,CD3-HLADR+NKG2D-CD8-CD45+CD4-CD5-CD14-CD19+CD1...
1,0_0_0_0_0,CD3+HLADR-NKG2D+CD8+CD45+CD4-CD5+CD14-CD19-CD1...
2,0_0_0_0_0,CD3+HLADR-NKG2D+CD8-CD45+CD4-CD5+CD14-CD19-CD1...
3,CD3-HLADR-NKG2D-CD8-CD45+CD4-CD5-CD14-CD19-CD1...,CD3-HLADR-NKG2D-CD8-CD45+CD4-CD5-CD14-CD19-CD1...
4,0_0_0_0_0,CD3+HLADR-NKG2D+CD8+CD45+CD4-CD5+CD14-CD19-CD1...


Finally, we're extracting the expression levels detected by FAUST.

Think of expression levels as simply a discretization of the "raw" expression values. E.g., if the protein's expression range is `[0,10]` we could choose to discretize the range into low and high, where low represents values in `[0,5]` and high represents values in `[5,10]`.

In [11]:
expression_levels = list(data.CD3_faust_annotation.unique())
expression_levels

['-', '+']

## Data Transformation

The following steps are the core of creating an annotation-driven embedding.

### Winsorize Data

We'll start by winsorizing the marker expressions one by one.

In [12]:
from scipy.stats.mstats import winsorize

winsorized_expression = raw_expression.copy()

for i, _ in enumerate(markers):
    winsorized_expression[:, i] = winsorize(winsorized_expression[:, i], limits=[0.01, 0.01])

### Normalize & Scale Data

Next, for _complete_ FAUST label (i.e., cluster) we are normalizing the expression values to have zero mean and unit variance (using [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)). And then we scale the normalized expression values of each marker (i.e., feature) to [0,1000] or [1000,2000] (using [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)) based on its expression level. In other words, we separate the expression ranges of each features based on the feature's expression level.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from time import time

std_scaler = StandardScaler()
expression_level_scalar = {
    '-': MinMaxScaler(feature_range=(0, 1000)),
    '+': MinMaxScaler(feature_range=(1000, 2000)),
}
expression_level_translation = {
    '-': 1000,
    '+': 2000,
}

faust_labels = data.complete_faust_label.unique()
num_faust_labels = len(faust_labels)

embedding_expression = winsorized_expression.copy()

t = 0

# 1. Normalize expressions for each phenotype to zero mean and unit variance
for i, faust_label in enumerate(faust_labels):
    if i % 1000 == 0:
        t = time()
        print(f'Transform {i}-{i + 999} of {len(faust_labels)} clusters... ', end='')
    
    # 1a. We get the indices of all data points belonging to the same complete FAUST label
    idxs = data.query(f'complete_faust_label == "{faust_label}"').index
    # 1b. Then we normalize the expression values of all the data points
    embedding_expression[idxs] = std_scaler.fit_transform(embedding_expression[idxs])

    # 2. Scale marker expressions based on the expression levels
    for m, marker in enumerate(markers):
        # 2a. Retrieve the expression level of a marker
        expression_level = mmrf_1267_embedding.iloc[idxs[0]][f'{marker}_faust_annotation']
        # 2b. Scale the normalized expression values of the marker to it's respective new feature range
        embedding_expression[idxs, m] = expression_level_scalar[expression_level].fit_transform(
            embedding_expression[idxs, m].reshape(-1, 1)
        ).flatten()
        # 2c. Further scale the normalized expression values respective new feature range
        embedding_expression[idxs, m] *= expression_level_translation[expression_level]
    
    if i % 1000 == 999:
        print(f'done! ({round(time() - t)}s)')

Transform 0-999 of 24721 clusters... done! (19s)
Transform 1000-1999 of 24721 clusters... done! (19s)
Transform 2000-2999 of 24721 clusters... 

### UMAP Embedding

The last step is to embed the data using UMAP (or any other kind of dimensionality reduction tool).

In [ ]:
from umap import UMAP

embedding = UMAP(n_neighbors=15, min_dist=0.2, metric='euclidean').fit_transform(embedding_expression)

Let's save the embedded data for easy access later on

In [ ]:
df_embedding = pd.concat(
    [mmrf_1267_embedding.faustLabels, pd.DataFrame(embedding, columns=['umapX', 'umapY'])],
    axis=1
)
df_embedding.faustLabels = df_new_embedding.faustLabels.astype('category')

df_embedding.to_parquet('data/MMRF_1267_embedding.pq', compression='gzip')

df_embedding.head()

## Visualize Embedding

In [24]:
# Uncomment the line below to load previously embedded data
# df_embedding = pd.read_parquet('data/MMRF_1267_embedding.pq')

scatter = jscatter.Scatter(data=df_embedding, x='umapX', y='umapY', height=640)
scatter.color(by='faustLabels', map=glasbey_dark+glasbey_dark+glasbey_dark)
scatter.show()